In [5]:
import os
import torch
import import_ipynb
import torch.nn as nn
import locations as l

    # A PyTorch implementation of a Multi-Layer Perceptron (MLP) model.

    # Attributes:
    #     layers (nn.ModuleList): A list of layers in the MLP, including both Linear layers and activation functions.

    # Methods:
    #     __init__(input_size, output_size, hidden_layers, activation_fn=nn.LeakyReLU):
    #         Initializes the MLP model with the specified input size, output size, hidden layers, and activation function.
    #     forward(x):
    #         Defines the forward pass of the MLP model.
    #     decode_model(model_path):
    #         Decodes the model architecture from the filename of a saved model.
    #     create_and_load(model_path, input_size, output_size):
    #         Creates an MLP model based on the architecture encoded in the filename and loads its weights from a file.

    # Usage:
    #     - This class is designed to create and manage MLP models with a flexible number of hidden layers and neurons.
    #     - It supports loading pre-trained models from a file.

    # Details:
    #     1. The number of hidden layers is limited to a maximum of 7 to prevent overly deep architectures.
    #     2. The activation function for the hidden layers can be customized (default is LeakyReLU).
    #     3. The model architecture can be encoded in the filename of a saved model, allowing for easy reconstruction.




class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, activation_fn=nn.ReLU):

        super(MLP, self).__init__()
        if len(hidden_layers) > 7:
            raise ValueError("The number of hidden layers cannot exceed 7.")

        self.layers = nn.ModuleList()
        prev_size = input_size

        # Create hidden layers
        for neurons in hidden_layers:
            self.layers.append(nn.Linear(prev_size, neurons))
            self.layers.append(activation_fn())  # Add the specified activation function
            prev_size = neurons

        # Create output layer
        self.layers.append(nn.Linear(prev_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    @staticmethod
    def decode_model_name(model_name):
        """
        Decodes a model name to extract the number of input neurons, output neurons, 
        hidden layer neurons, names of output variables, and input variables.
    
        Args:
            model_name (str): The model name to decode.
    
        Returns:
            dict: A dictionary containing the decoded information.
        """
        # Remove the file extension
        model_name =os.path.basename(model_name)

        model_name = model_name.replace(".pth", "")
    
        # Split the model name into parts
        parts = model_name.split("_")
        #print(parts)
        # Extract input and output variable names
        input_start = parts.index("model") + 1
        to_index = parts.index("to")
        feature_columns = parts[input_start:to_index]
    
        output_start = to_index + 1
        standardization_index = parts.index("standardized") if "standardized" in parts else parts.index("non_standardized")
        target_columns = parts[output_start:standardization_index]
    
        # Extract hidden layers
        layers_index = parts.index("layers") + 1
        hidden_layers = list(map(int, parts[layers_index:]))
    
        # Calculate the number of input and output neurons
        input_neurons = len(feature_columns)
        output_neurons = len(target_columns)
    
        # Return the decoded information
        return {
            "input_neurons": input_neurons,
            "output_neurons": output_neurons,
            "hidden_layers": hidden_layers,
            "input_variables": feature_columns,
            "output_variables": target_columns
        }

    @staticmethod
    def create_and_load(model_name,input_size, output_size):
        #print(f"Loading model from {model_path}")
        models_dir = l.locations.get_models_dir()
        model_path = os.path.join(models_dir, model_name)
        data = MLP.decode_model_name(model_name)
        hidden_layers = data["hidden_layers"]
        model = MLP(input_size, output_size, hidden_layers)
        if torch.cuda.is_available():
            model.load_state_dict(torch.load(model_path))
        else:
            model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        return model

